In [ ]:
import obsidian
obsidian.__version__

In [ ]:
import pandas as pd
import plotly.express as px

In [ ]:
from obsidian.parameters import ParamSpace, Param_Categorical, Param_Ordinal, Param_Continuous
from obsidian.experiment import ExpDesigner

### Set up parameter space and initialize a design

In [ ]:
params = [
    Param_Continuous('Temperature', -10, 30),
    Param_Continuous('Concentration', 10, 150),
    Param_Continuous('Enzyme', 0.01, 0.30),
    Param_Categorical('Variant', ['MRK001', 'MRK002', 'MRK003']),
    Param_Ordinal('Stir Rate', ['Low', 'Medium', 'High']),
    ]

X_space = ParamSpace(params)
designer = ExpDesigner(X_space, seed=0)
X0 = designer.initialize(10, 'LHS')

X0

### Collect results (e.g. from a simulation)

In [ ]:
from obsidian.experiment import Simulator
from obsidian.experiment.benchmark import shifted_parab

simulator = Simulator(X_space, shifted_parab, name='Yield', eps=0.05)
y0 = simulator.simulate(X0)
Z0 = pd.concat([X0, y0], axis=1)

Z0

In [ ]:
Z0.to_csv('Example APO Data Input.csv')

In [ ]:
Z0.plot(x='Temperature', y='Yield', kind='scatter', figsize=(4,3))

### Create a campaign to track optimization

In [ ]:
from obsidian.campaign import Campaign

In [ ]:
my_campaign = Campaign(X_space)
my_campaign.add_data(Z0)
my_campaign.data

### Fit an optimizer

In [ ]:
from obsidian.parameters import Target

In [ ]:
target = Target('Yield', aim='max')
my_campaign.set_target(target)
my_campaign.fit()

### Make new experiment suggestions

In [ ]:
X_suggest, eval_suggest = my_campaign.optimizer.suggest(m_batch=1, acquisition = ['EI','EI'])

In [ ]:
df_suggest = pd.concat([X_suggest, eval_suggest], axis=1)
df_suggest

### Collect data at new suggestions

In [ ]:
y_iter1 = pd.DataFrame(simulator.simulate(X_suggest), columns = ['Yield'])
Z_iter1 = pd.concat([X_suggest, y_iter1, eval_suggest], axis=1)
my_campaign.add_data(Z_iter1)

### Repeat as desired

In [ ]:
Z_iter1.reset_index(drop=True)

In [ ]:
my_campaign.data

In [ ]:
for iter in range(3):
    my_campaign.fit()
    X_suggest, eval_suggest = my_campaign.optimizer.suggest(m_batch=3)
    y_iter = pd.DataFrame(simulator.simulate(X_suggest))
    Z_iter = pd.concat([X_suggest, y_iter, eval_suggest], axis=1)
    my_campaign.add_data(Z_iter)

In [ ]:
my_campaign.data

In [ ]:
fig = px.scatter(my_campaign.data, x=my_campaign.data.index, y='Yield', color='aq Value')
fig.update_layout(height=300, width=400, template='ggplot2')

In [ ]:
my_campaign.data